In [ ]:
import numpy as np
from pydrake.examples.manipulation_station import ManipulationStation

from pydrake.all import (
    DiagramBuilder, MeshcatVisualizerCpp, MeshcatVisualizerParams, 
    Simulator, FindResourceOrThrow,
    Parser, MultibodyPlant, RigidTransform, RollPitchYaw,
    PiecewisePolynomial, PiecewiseQuaternionSlerp, RotationMatrix, Solve,
    TrajectorySource, ConstantVectorSource
)
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial
from manipulation.meshcat_cpp_utils import (
  StartMeshcat, AddMeshcatTriad
)
from manipulation.scenarios import AddMultibodyTriad

## Shamelessly Copying from 8.1

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

Meshcat is now available at https://94a9bd61-ed58-4c78-95db-bc94494e48ba.deepnoteproject.com


In [ ]:
def setup_manipulation_station():
  builder = DiagramBuilder()
  station = builder.AddSystem(ManipulationStation(time_step=1e-3))
  station.SetupManipulationClassStation()
  AddMultibodyTriad(
      station.get_multibody_plant().GetFrameByName("body"), station.get_scene_graph())
  station.Finalize()

  MeshcatVisualizerCpp.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False))

  wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
  builder.Connect(wsg_position.get_output_port(),
                  station.GetInputPort("wsg_position"))

  diagram = builder.Build()

  plant = station.get_multibody_plant()
  context = plant.CreateDefaultContext()
  gripper = plant.GetBodyByName("body")

  initial_pose = plant.EvalBodyPoseInWorld(context, gripper)

  simulator = Simulator(diagram)
  simulator.set_target_realtime_rate(1.0)
  simulator.AdvanceTo(0.01)

  return initial_pose

# Get initial pose of the gripper by using default context of manip station.
initial_pose = setup_manipulation_station()

p_WR = np.array([0.7477, -0.1445, 0.4148]) # frame R: center of left door.

p_Rhandle = np.array([-0.033, 0.1245, 0]) # handle: frame attached to right handle.
p_Whandle = p_WR + p_Rhandle

p_Rhinge = np.array([0.008, -0.1395, 0]) # hinge: frame attached to right hinge.
p_Whinge = p_WR + p_Rhinge

p_Rhinge_handle = p_Rhandle - p_Rhinge
r_Rhinge_handle = np.linalg.norm(p_Rhandle - p_Rhinge) # distance between handle and hinge.

theta_Rhinge_handle = np.arctan2(p_Rhinge_handle[1], p_Rhinge_handle[0])
angle_end = np.pi # end of angle. Decrease to 120~160 deg for the easy version.

# Interpolate pose for opening doors.
def InterpolatePoseOpen(t):
  # Start by interpolating the yaw angle of the hinge.
  angle_start = theta_Rhinge_handle
  theta = angle_start + (angle_end - angle_start) * t
  # Convert to position and rotation.
  p_Whandle = r_Rhinge_handle * np.array([np.cos(theta), np.sin(theta), 0]) + p_Whinge
  # Add some offset here to account for gripper yaw angle.
  R_Whandle = RollPitchYaw(0, 0, theta).ToRotationMatrix()
  X_Whandle = RigidTransform(R_Whandle, p_Whandle)

  # Add a little offset to account for gripper.
  p_handleG = np.array([0., 0.1, 0.])
  R_handleG = RollPitchYaw(0, np.pi, np.pi).ToRotationMatrix()
  X_handleG = RigidTransform(R_handleG, p_handleG)
  X_WG = X_Whandle.multiply(X_handleG)
  return X_WG

## Interpolate Pose for entry.
def make_gripper_orientation_trajectory():
  traj = PiecewiseQuaternionSlerp()
  traj.Append(0.0, initial_pose.rotation())
  traj.Append(5.0, InterpolatePoseOpen(0.0).rotation())
  return traj

def make_gripper_position_trajectory():
  traj = PiecewisePolynomial.FirstOrderHold(
      [0.0, 5.0],
      np.vstack([[initial_pose.translation()],
                 [InterpolatePoseOpen(0.0).translation()]]).T)
  return traj

entry_traj_rotation = make_gripper_orientation_trajectory()
entry_traj_translation = make_gripper_position_trajectory()

def InterpolatePoseEntry(t):
  return RigidTransform(RotationMatrix(entry_traj_rotation.value(t)),
                        entry_traj_translation.value(t))

# Wrapper function for end-effector pose. Total time: 11 seconds.
def InterpolatePose(t):
  if (t < 5.0):
    # Duration of entry motion is set to 5 seconds.
    return InterpolatePoseEntry(t)
  elif (t >= 5.0) and (t < 6.0):
    # Wait for a second to grip the handle.
    return InterpolatePoseEntry(5.0)
  else:
    # Duration of the open motion is set to 5 seconds.
    return InterpolatePoseOpen((t - 6.0) / 5.0)

# Visualize our end-effector nominal trajectory.
t_lst = np.linspace(0, 11, 30)
pose_lst = []
for t in t_lst:
  AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePose(t), opacity=.2)
  pose_lst.append(InterpolatePose(t))

# Create gripper trajectory.
gripper_t_lst = np.array([0., 5., 6., 11.])
gripper_knots = np.array([0.05, 0.05, 0., 0.]).reshape(1,4)
g_traj = PiecewisePolynomial.FirstOrderHold(gripper_t_lst, gripper_knots)

def CreateIiwaControllerPlant():
    """creates plant that includes only the robot and gripper, used for controllers."""
    robot_sdf_path = FindResourceOrThrow(
        "drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
    gripper_sdf_path = FindResourceOrThrow(
        "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf")
    sim_timestep = 1e-3
    plant_robot = MultibodyPlant(sim_timestep)
    parser = Parser(plant=plant_robot)
    parser.AddModelFromFile(robot_sdf_path)
    parser.AddModelFromFile(gripper_sdf_path)
    plant_robot.WeldFrames(
        frame_on_parent_P=plant_robot.world_frame(),
        frame_on_child_C=plant_robot.GetFrameByName("iiwa_link_0"))
    plant_robot.WeldFrames(
        frame_on_parent_P=plant_robot.GetFrameByName("iiwa_link_7"),
        frame_on_child_C=plant_robot.GetFrameByName("body"),
        X_PC=RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), np.array([0, 0, 0.114]))
    )
    plant_robot.mutable_gravity_field().set_gravity_vector([0, 0, 0])
    plant_robot.Finalize()

    link_frame_indices = []
    for i in range(8):
        link_frame_indices.append(
            plant_robot.GetFrameByName("iiwa_link_" + str(i)).index())

    return plant_robot, link_frame_indices

def BuildAndSimulateTrajectory(q_traj, g_traj):
  """Simulate trajectory for manipulation station.
  @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
  @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
  """
  builder = DiagramBuilder()
  station = builder.AddSystem(ManipulationStation(time_step=1e-3))
  station.SetupManipulationClassStation()
  AddMultibodyTriad(
      station.get_multibody_plant().GetFrameByName("body"), station.get_scene_graph())
  station.Finalize()

  q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
  g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

  MeshcatVisualizerCpp.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False))

  builder.Connect(q_traj_system.get_output_port(),
                  station.GetInputPort("iiwa_position"))
  builder.Connect(g_traj_system.get_output_port(),
                  station.GetInputPort("wsg_position"))

  diagram = builder.Build()

  simulator = Simulator(diagram)
  simulator.set_target_realtime_rate(1.0)
  simulator.AdvanceTo(0.01)

  station_plant = station.get_multibody_plant()

  return simulator, station_plant

## make balls, apply physics to them

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=94a9bd61-ed58-4c78-95db-bc94494e48ba' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>